原文： http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

Translator: 陳胤泰 YinTaiChen

神經網路 (Neural Networks)
===============

使用``torch.nn``套組可以建造出神經網路。

既然你已經對``autograd``有了初步的了解，``nn``仰賴``autograd``定義神經網路模型 (model) 並使模型各有不同。

一個``nn.Module``物件包含神經網路層（layer）與回傳``output``的``forward(input)``方法。

舉例來說，看看這個分類數位影像的網路：

![title](http://pytorch.org/tutorials/_images/mnist.png)
convnet

這是個簡單的向前傳遞（feed-forward）網路。它會傳遞輸入給神經網路層，一層接著一層，直到最終輸出結果。

訓練神經網路的典型程序如下：

- 定義一個神經網路，該網路具有可學習的參數（或稱為權重）
- 遞迴經歷一個作為輸入的資料集
- 對輸入做運算，直到通過整個神經網路
- 計算損失（輸出有多接近正確結果）
- 將梯度反向傳播成神經網路的參數
- 更新神經網路的權重值，以這個簡單的規則為代表：
   ``權重 ＝ 權重 - 學習率 * 梯度``


定義網路
------------------
讓我們定義一個像這樣的網路：

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


你只須定義``forward``函式。``backward``函式（梯度於此被計算）會在使用``autograd``的同時，自動被定義好。

在``forward``函式中，你可以使用任何的張量運算。

模型可學習的參數由``net.parameters()``函式回傳。

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


要向前傳遞的輸入是個``autograd.Variable``物件，輸出也是。

這個網路（LeNet）預期的輸入大小為 32x32。

注意：若要將這個網路用在 MNIST 資料集，請將該資料集的影像大小縮放至 32x32。

In [3]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
 0.0132 -0.0327 -0.0120  0.0312  0.0905 -0.0440  0.0449 -0.0883 -0.0292  0.1113
[torch.FloatTensor of size 1x10]



將所有參數的梯度暫存區 (gradient buffer) 歸零，並以隨機梯度做反向傳播：

<div class="alert alert-info"><h4>注意事項</h4><p>torch.nn套組只支援小批次學習（mini-batch，亦即它只接受小批的資料樣本（sample），而非單一資料樣本。
    
    舉例來說，``nn.Conv2d``會接收4D張量，其維度為：``樣本數 x 頻道數 x 高度 x 寬度``。

    如果你有一個單獨的資料樣本，只須使用``input.unsqueeze(0)``函式來添加一個假的批次維度（batch dimension）。</p></div>

在更進一步之前，讓我們複習一下到目前為止見過的所有類別。

**複習:**
  -  ``torch.Tensor`` - 『多維陣列』的類別。
  -  ``autograd.Variable`` - 將張量包裹起來，並紀錄對其所做的所有運算。與``Tensor``有相同的應用程式界面(API)。它也具有與張量相關聯的的梯度。
  -  ``nn.Module`` - 神經網路模組。是個封裝(encapsulating)參數的簡便方法，並幫助使用者將參數移動到GPU、匯出、匯入等等。
  -  ``nn.Parameter`` - Variable 的一種，唯其在被指派為（assigned）``Module``的屬性時，被自動登記為參數。
     ``Module``.
  -  ``autograd.Function`` - 實作 autograd 運算向前與向後的定義。每一個``Variable``運算會創建至少一個``Function``節點(node)。該節點與創建``Variable``的函式相連，並編碼了``Variable``的創建歷史。

**至此，我們涵蓋了：**
  -  定義一個神經網路
  -  對輸入做運算並實施反向傳播

**還剩下:**
  -  計算損失
  -  更新神經網路的權重

損失函數 (Loss Function)
-------------
損失函數接收（輸出值, 目標值）這樣一對數值作為它的輸入，並計算出一個數值，用來估計輸出值與目標值相距多遠。

在 nn 套組之中，有許多不同的`loss function`。

``nn.MSELoss``是個簡單的例子。它計算輸出值與目標值之間的平均方差(mean-squared error)。

舉例來說:



In [4]:
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.3670
[torch.FloatTensor of size 1]



現在呢，如果你循著反向傳播的方向，並使用``loss``的``.grad_fn``屬性來追蹤它，你會看到像這樣一張計算圖(graph of computations)：

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

當我們呼叫``loss.backward()``，整張圖將對 loss 做微分。圖中所有的 Variable 將得到``.grad``這個 Variable，並隨著梯度累積。

為了具體的了解，讓我們追蹤幾個反向傳播的步驟：

In [5]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向傳播
--------
要反向傳播誤差，我們只需呼叫``loss.backward()``。

然而，你得清除既存的梯度，否則計算出來的梯度會累積它之上。

如此我們便能呼叫``loss.backward()``，並看看第一層卷積層之偏權值(bias)的梯度，在反向傳播之前與之後的變化。

In [6]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
Variable containing:
 0.1114
-0.0492
 0.0608
-0.0078
 0.0347
-0.0627
[torch.FloatTensor of size 6]



至此，我們了解到該如何使用損失函數。

**稍後閱讀:**

  與神經網路相關的套組具有各式各樣的模組與損失函數，這兩者形成了建構深層神經網路(deep neural networks)的模塊。
  
  完整的清單與說明文件在此：http://pytorch.org/docs/nn

**只剩一件事還沒學:**

  - 更新神經網路的權重

更新權重
------------------
在實際應用上，更新權重最簡單的規則是梯度隨機下降法(Stochastic Gradient Descent; SGD)：

     梯度 = 梯度 - 學習率 * 梯度

我們可以用簡單幾行 Python 程式碼，將這條規則實作出來。

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

然而，在使用神經網路時，你會想要使用各式各樣的更新規則，像是SGD、Nesterov-SGD、Adam、RMSProp，等等。

為了讓這件事成真，我們建造了一個小套組：``torch.optim``。

它實作了上述所有方法。使用它是很簡單的：

In [7]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update